In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Introducción a Machine Learning - Aprendizaje no supervisado

## Tabla de Contenidos

1. <a href="#section_introduccion">Introducción</a>
2. <a href="#section_no_supervisado">Aprendizaje no supervisado</a>
- 2.1 <a href="#section_clustering">Clustering</a>
- 2.2 <a href="#section_reduccion">Reducción de la dimensionalidad</a>
3. <a href='#section_recap'>Recapitulando</a>
---

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

<a id="section_introduccion"></a>
## Introducción
<img src="img/sklearn.png" width="450">
<br>

Ya hemos visto ejemplos sencillos de problemas de aprendizaje supervisado, donde buscamos predecir a partir de ciertas _features_ una variable _target_ que puede ser continua (problemas de regresión) o categórica (clasificación). Tuvimos un primer contacto con la biblioteca Scikit-Learn, la herrramienta por excelencia para trabajar con _machine learning_ en Python.

**En esta notebook, exploraremos el aprendizaje no supervisado**.

<a id="section_no_supervisado"></a>
## Aprendizaje no supervisado

<div id="caja9" style="float:left;width: 85%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>Comencemos repasando en qué consiste este otro tipo de <i>machine learning</i><br></label></div>

El aprendizaje no supervisado **se caracteriza por prescindir de una variable _target_. El foco está puesto en descubrir patrones o estructuras subyacentes en la información que nos permitan generar _insights_ relevantes en una estructura de datos que, a priori, no son evidentes.** Dentro del aprendizaje no supervisado, se engloban las técnicas de **_clustering_** y las de **reducción de la dimensionalidad**.<br>

Veremos un ejemplo de cada una de ellas a partir del dataset de [Iris](https://es.wikipedia.org/wiki/Conjunto_de_datos_flor_iris), sobre el cual ya estuvimos trabajando anteriormente para entrenar nuestro primer modelo de clasificación.

<img src="https://shahinrostami.com/images/ml-with-kaggle/iris-2338142_960_720-1.jpg">
<center><i>La flor iris</i></center>

Podemos cargar el dataset desde la librería Seaborn:

In [ ]:
iris = sns.load_dataset('iris')
iris.head()

Inspeccionamos la tabla y describimos la información:

In [ ]:
iris.info()

In [ ]:
iris.describe()

Visualizamos los datos:

In [ ]:
sns.pairplot(iris, hue='species');

<div id="caja10" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <br>
  <br>
  <div style="float:left;width: 85%;"><label><b>¿Qué apreciamos en estos gráficos? ¿Existen patrones relevantes que surjan a simple vista? ¿Se aprecian grupos de observaciones distinguibles fácilmente entre sí?</b></label></div>
</div>

<a id="section_clustering"></a>
### Clustering

Los algoritmos de _clustering_ nos permiten segmentar las observaciones asignándolas automáticamente a distintos grupos o _clusters_, de forma de que observaciones similares se encuentren agrupadas y que los grupos formados sean más bien distintos entre sí. Algunas aplicaciones frecuentes de este tipo de análisis son la segmentación de una cartera de clientes de una compañía o la identificación de expresiones genéticas similares en estudios biológicos.

En este caso, vamos a aplicar una **técnica clásica de _clustering_** llamada **K-Means** al dataset de Iris. Este algoritmo trata de encontrar grupos de observaciones más bien homogéneas entre sí sin tener referencias a etiquetas en los datos. La cantidad de _clusters_ que obtenemos debe ser especificada de antemano. Vamos a analizar si los grupos o _clusters_ que obtenemos responden a las distintas especies -como esperaríamos- o no.

El algoritmo de K-Means se puede resumir básicamente en una serie de pasos:

1. Asignar aleatoriamente los puntos centrales (centroides) de los _k clusters_ que se desean obtener.
2. Calcular la distancia de las observaciones a estos puntos centrales y asignar cada muestra al _cluster_ cuyo centroide resulta ser el más cercano.
3. Recalcular la posición del punto central usando la asignación actual de observaciones de cada _cluster_.
4. Iterar los pasos 2 y 3 hasta alcanzar algún criterio de convergencia.

<div id="caja10" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_saber_mas.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;"><label>Si bien no ahondaremos en demasiados detalles, <a href="http://shabal.in/visuals/kmeans/left.gif">esta animación</a> nos permite generar una intuición visual acerca del comportamiento de <b>K-Means</b>.</label></div>
</div>

Notemos que, pese a tratarse de un problema no supervisado, el paso a paso con Scikit-Learn que seguiremos a continuación es prácticamente igual al que hemos visto anteriormente:

##### **1. Seleccionamos una clase de modelo**

In [ ]:
from sklearn.cluster import KMeans

##### **2. Elegir los hiperparámetros del modelo**

In [ ]:
model = KMeans(n_clusters=3, random_state=0) # n_clusters define cuántos clusters queremos obtener y random_state controla la aleatoriedad inicial

##### **3. Preparar los datos en una matriz de _features_**

Como se trata de un problema no supervisado, por definición, no contaremos con una variable objetivo. Por eso, construimos la matriz de _features_ descartando la variable species.

In [ ]:
X_iris = iris.drop('species', axis=1)
X_iris.shape

Dado que K-Means es un algoritmo que trabaja calculando distancias, un paso previo al ajuste del modelo consiste en **estandarizar los valores**, de forma de eliminar posibles inconvenientes asociados a la escala de las distintas variables. Éste es un paso de **preprocesamiento de los datos** que será frecuente en muchas de las aplicaciones que estudiaremos en el curso.

Numerosas herramientas de preprocesamiento se encuentran en el módulo de [`preprocessing`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) de Scikit-Learn. De allí, importamos la clase **`StandardScaler()`**, un **transformador** de los datos que reescala las variables para que tengan **media 0 y desvío estándar 1**.

In [ ]:
from sklearn.preprocessing import StandardScaler

Al tratarse de una clase, también es necesario instanciarla en un objeto:

In [ ]:
scaler = StandardScaler()

Los transformadores también se ajustan a los datos, al igual que los estimadores:

In [ ]:
scaler.fit(X_iris)

A diferencia de los estimadores, en lugar del método `predict()`, los transformadores tienen el método `transform()`:

In [ ]:
scaler.transform(X_iris)[:5]

Asignamos los resultados a una matriz X:

In [ ]:
X = scaler.transform(X_iris)
print('Medias reescaladas:', np.mean(X, axis=0).round(2))
print('Desvíos reescalados:', np.std(X, axis=0).round(2))

Como el fin último de este tipo de análisis es generar grupos de observaciones únicamente a partir de sus características, sin tener presente ningún tipo de variable objetivo, en los problemas de _clustering_ (y de aprendizaje no supervisado en general) no es necesario hacer la separación de los conjuntos de entrenamiento y testeo. Por eso, en este punto, ya estamos listos para hacer el ajuste del modelo.

##### **4. Ajustar el modelo a los datos**

In [ ]:
model.fit(X)

<div id="caja10" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/haciendo_foco.png" style="align:center"/> </div>
  <br>
  <br>
  <div style="float:left;width: 85%;"><label><b>Notar que no especificamos ninguna variable $y$.</b></label></div>
</div>

##### **5. Predecir etiquetas**

En este caso, la predección consiste en generar etiquetas que identifiquen cada observación con un _cluster_ en particular.

In [ ]:
y_km = model.predict(X)
y_km

In [ ]:
# Visualizamos los resultados
X_iris['cluster'] = y_km
sns.pairplot(X_iris, hue='cluster');

Comparando los dos _pair plot_ que hemos graficado, podemos observar que la segmentación de flores resultante del análisis de _clustering_ guarda una estrecha relación con las etiquetas reales, incluso aunque K-Means jamás las observó durante el ajute a los datos.

Esto significa que aun sin un experto que nos indique las etiquetas de las flores individuales, las _features_ de estas observaciones son lo suficientemente distintivas para que podamos identificar *automáticamente* las diferentes especies con un simple algoritmo de _clustering_.

<a id="section_reduccion"></a>
### Reducción de la dimensionalidad

Las técnicas de reducción de la dimensionalidad buscan obtener a partir de un dataset complejo, con numerosas variables (alta dimensionalidad), una representación en una menor cantidad de variables (baja dimensionalidad) que de alguna manera preserve cualidades relevantes del dataset original. Estas técnicas son muy utilizadas en la etapa de análisis exploratorio de los datos para poder generar visualizaciones interesantes de todas las variables en tan sólo dos o tres dimensiones. También, muchas veces se las utiliza como parte del preprocesamiento de los datos, de forma de obtener nuevas _features_ que alimenten a otro tipo de algoritmos supervisados o no supervisados.

Como un ejemplo de un problema de aprendizaje no supervisado, veamos cómo reducir la dimensionalidad de los datos del dataset Iris para poder visualizarlos más fácilmente.

Como hemos visto, el dataset consta de 150 muestras de 3 especies de la flor Iris, de las cuales disponemos de 4 atributos: el largo y ancho del pétalo y del sépalo.

Por lo tanto, el dataset Iris es cuatridimensional: hay 4 _features_ medidas para cada observación (*sample*).

La tarea de reducción de la dimensionalidad es investigar si hay una representación apropiada de baja dimensionalidad que retiene las características esenciales de los datos originales. 

En este ejemplo, vamos a usar **Principal Component Analysis (PCA)**, una técnica rápida de reducción lineal de la dimensionalidad. Cada componente obtenido es una combinación lineal de todas las variables, donde algunas tienen mayor peso que otras según el caso.

Siguiendo la secuencia de pasos presentada previamente, tenemos:

##### **1. Seleccionamos una clase de modelo**

PCA trabaja con métodos de descomposición de matrices, de ahí que se encuentre en el módulo [`decomposition`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) de Scikit-Learn.

In [ ]:
from sklearn.decomposition import PCA  

##### **2. Elegir los hiperparámetros del modelo**

Vamos a pedirle al modelo que devuelva dos componentes, es decir, una representación bidimensional de los datos originales.

In [ ]:
model = PCA(n_components=2)            

##### **3. Preparar los datos en una matriz de _features_**

Como PCA requiere un preprocesamiento similar a K-Means, en este punto podemos reutilizar nuestra matriz X anteriormente definida.

In [ ]:
X[:5]

##### **4. Ajustar el modelo a los datos**

In [ ]:
model.fit(X)       

##### **5. Predecir etiquetas**

En este caso, la predección consiste en generar los componentes principales a partir de las variables originales. Al igual que en el caso de `StandardScaler`, cuando trabajamos con modelos de reducción de la dimensionalidad, esto se logra con el método `transform()`:

In [ ]:
X_2D = model.transform(X)
X_2D[:5]

Ahora vamos a plotear los resultados. Una forma rápida de hacer esto es insertar los resultados en el ``DataFrame`` original de Iris, y usar el método ``lmplot`` de Seaborn para mostrar los resultados:


In [ ]:
iris['PCA1'] = X_2D[:, 0]
iris['PCA2'] = X_2D[:, 1]
sns.lmplot("PCA1", "PCA2", hue='species', data=iris, fit_reg=False);

Vemos que en la representación en dos dimensiones, las especies están relativamente bien separadas... ¡incluso aunque el algoritmo PCA no tenía conocimiento de las etiquetas de las especies de flores!

Esto significa que es posible condensar la información de las cuatro _features_ originales en dos variables nuevas que se construyen a partir de todas ellas y que buscan preservar la esencia de la información que están condensando.

<a id="section_recap"></a>
## Recapitulando

Hemos tenido nuestro primer contacto con aplicaciones de _machine learning_ no supervisado con Scikit-Learn. Este tipo de aprendizaje se caracteriza por la ausencia de una variable objetivo. El objetivo del aprendizaje no supervisado consiste en descubrir patrones o estructuras subyacentes en la información que nos permitan **segmentar las observaciones (_clustering_) o generar representaciones de los datos en una menor dimensionalidad**. Trabajamos con el algoritmo **K-Means** para identificar _clusters_ de flores Iris y y utilizamos **PCA** para generar dos nuevas variables que nos permitieron representar en sólo dos dimensiones la información intrínseca de las cuatro _features_ registradas para cada una de las flores.

<div id="caja10" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/en_resumen.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;"><label><b>A esta altura, ya hemos cubierto a grandes rasgos los distintos tipos de aprendizajes posibles, las características esenciales de la representación de datos y el flujo de trabajo típico de Scikit-Learn.</b></label></div>
</div>



Sin importar el tipo de estimador, el mismo patrón de importar-instanciar-ajustar-predecir se mantiene en todos los casos, aunque con algunas diferencias que van a depender del tipo de problema con el que estemos trabajando.

Con estas nuevas herramientas, ya podés explorar la documentación de Scikit-Learn y comenzar a probar varios modelos sobre tus datos, teniendo siempre presente qué tipo de pregunta deseás responder:

<div id="caja10" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/ponete_a_prueba.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label><b>- ¿Tengo una variable objetivo o pretendo hallar estructuras suyacentes en los datos para enriquecer el análisis?<br/>
- ¿Quiero predecir un valor continuo o categórico?<br/>
- ¿Deseo segmentar las observaciones?<br/>
- ¿Cómo puedo generar visualizaciones que condensen la esencia de los datos?</b></label></div>
</div>